# Computational Phase Retrieval with Tensor Methods

## Device Information

In [1]:
!nvidia-smi

Sun Jul 18 17:21:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8     6W /  N/A |    344MiB /  3911MiB |     20%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import Required Libraries

In [2]:
import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        print("Num GPUs Available: ", len(gpus))
        for gpu in gpus:
            # Allow memory growth for the GPU.
            # Reference: https://www.tensorflow.org/guide/gpu
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized.
        print(e)

from matplotlib import pyplot as plt
plt.style.use('dark_background')
import numpy as np
import tensorly as tl
tl.set_backend('tensorflow')
import cv2
import time
import os
from PIL import Image

import matplotlib.cm as cm
import seaborn as sns

from matplotlib import rc, rcParams
rcParams['font.family'] = 'Cubano'
rc('text', usetex=False)

import warnings
from numba import jit, njit, prange
from scipy.optimize import minimize
import functools


Tensorflow version: 2.5.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


## GPU Benchmark

Run preliminarily to avoid cold-start.

Reference: https://www.tensorflow.org/guide/gpu

In [6]:
tf.debugging.set_log_device_placement(True)

n = 1000
num_iters = 10

'''
Test with TensorFlow GPU.
'''
start_tf = time.time()

for i in range(num_iters):
    # Tensors are defaultly placed on the GPU (CPU would be considerably 
    # slower due to the incurred communication cost).
    a = tf.ones((n, n))
    b = tf.ones((n, n))

    # Run on the GPU
    c = tf.matmul(a, b)

print(f'Elapsed time with TensorFlow GPU: {time.time() - start_tf}')

'''
Test with Numpy.
'''
start_np = time.time()

for i in range(num_iters):
    a = np.ones((n, n))
    b = np.ones((n, n))

    c = np.dot(a, b)

print(f'Elapsed time with Numpy: {time.time() - start_np}') # CAN BE SLOW


Elapsed time with TensorFlow GPU: 0.0034453868865966797
Elapsed time with Numpy: 0.26870226860046387


## Low Rank Phase Retrieval

References:

\[1\] Namrata Vaswani, Seyedehsara Nayer, Yonina C. Eldar. *Low Rank Phase Retrieval*. https://rutgers.box.com/s/dntl0sh157p62rgi1zerdaxrqthugr32

\[2\] Namrata Vaswani. *Nonconvex Structured Phase Retrieval*. https://rutgers.box.com/s/x02w8frd1ep01cxdjlnojufa9npvstsz.

\[3\] Tamara G. Kolda, Brett W. Bader. *Tensor Decompositions and Applications*. https://rutgers.box.com/s/aq9psx3mgwhms6rrzlhn94h56c3oshox. 




### Define Data Directories

In [7]:
INPUT_DIR = './videos/' # directory of the test videos
OUTPUT_DIR = './output/' # output directory
FRAMES_DIR = './ouput/frames/' # output directory of the extracted video frames 

### Load the Test Video

In [8]:
# Read the video.
video_path = INPUT_DIR + os.listdir(INPUT_DIR)[0] # define video path
cap = cv2.VideoCapture(video_path) # read the video from path
video_name = os.listdir(INPUT_DIR)[0].split('.')[0] # get the name of the video

# Creat the folder to store the extracted frames of the video.
try:
    if not os.path.exists(FRAMES_DIR + video_name):
        os.makedirs(FRAMES_DIR + video_name)
    else:
        print('Directory already exists!')
except OSError:
    print('OS ERROR')

k = 0 # frame number, k = 0, 1, 2, ..., q - 1
Xlist = []
Rhat = 0
while (True):
    # Capture the video frame-by-frame.
    # Code adopted: https://docs.opencv.org/3.4/dd/d43
    # tutorial_py_video_display.html
    ret, frame = cap.read()

    # If the frame is read correctly the return boolean (ret) is true.
    if not ret:
        print("Cannot receive frame (probably end of stream). Exiting...")
        break
    else:
        # Convert the frame to grayscale.
        gray_frame_original = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        scale = 0.025
        width = int(gray_frame_original.shape[1] * scale)
        height = int(gray_frame_original.shape[0] * scale)
        gray_frame = cv2.resize(gray_frame_original, (width, height))
        name = FRAMES_DIR + video_name + '/frame-' + str(k) + '.jpg'
        print('DEBUG: Captured...' + name)
        svds = np.linalg.svd(gray_frame)[1]
        max_svd, min_svd = np.max(svds), np.min(svds)
        normalized_svds = svds / (max_svd - min_svd)
        Rhat += np.sum(normalized_svds > 0.1)
        cv2.imwrite(name, gray_frame)

        # plt.plot(range(480), normalized_svds)
        # plt.show()
        
        Xlist.append(gray_frame)

        k += 1
Rhat = Rhat // k + 1
print('Approximate rank of each frame: ', Rhat)

# Release the capture when finished.
cap.release()
cv2.destroyAllWindows()

Directory already exists!
DEBUG: Captured..../ouput/frames/sara/frame-0.jpg
DEBUG: Captured..../ouput/frames/sara/frame-1.jpg
DEBUG: Captured..../ouput/frames/sara/frame-2.jpg
DEBUG: Captured..../ouput/frames/sara/frame-3.jpg
DEBUG: Captured..../ouput/frames/sara/frame-4.jpg
DEBUG: Captured..../ouput/frames/sara/frame-5.jpg
DEBUG: Captured..../ouput/frames/sara/frame-6.jpg
DEBUG: Captured..../ouput/frames/sara/frame-7.jpg
DEBUG: Captured..../ouput/frames/sara/frame-8.jpg
DEBUG: Captured..../ouput/frames/sara/frame-9.jpg
DEBUG: Captured..../ouput/frames/sara/frame-10.jpg
DEBUG: Captured..../ouput/frames/sara/frame-11.jpg
DEBUG: Captured..../ouput/frames/sara/frame-12.jpg
DEBUG: Captured..../ouput/frames/sara/frame-13.jpg
DEBUG: Captured..../ouput/frames/sara/frame-14.jpg
DEBUG: Captured..../ouput/frames/sara/frame-15.jpg
DEBUG: Captured..../ouput/frames/sara/frame-16.jpg
DEBUG: Captured..../ouput/frames/sara/frame-17.jpg
DEBUG: Captured..../ouput/frames/sara/frame-18.jpg
DEBUG: Captured

### Create the true signal tensor.

Tensors are multi-dimensional arrays with a uniform type (`dtype`). All tensors are immutable like Python numbers and strings: you can never update the contents of a tensor, only create a new one.

**Note**: In libraries like tensorflow, the rank of the tensor actually denotes the order of the tensor in our convention. We call the `rank` of a tensor in a similar manner as the rank of a matrix.

The gray-scaled signal is modeled as a three-ordered tensor $\boldsymbol{\mathcal{X}} \in \mathbb{R}^{I_1 \times I_2 \times q}$, where $I_1 \times I_2$ correspond to the pixel coordinates within each frame and $q$ is the total number of frames captured.

**Signal Dimension**

In [9]:
Xast = tf.constant(Xlist[:10], tf.float32)
q, I1, I2 = Xast.shape
Xast = tf.reshape(Xast, [I1, I2, q])
print(f'The dimension of the true signal tensor: I1 x I2 x q: {I1} x {I2} x {q}')
print(f'Sample complexity for rank {Rhat}: O({(q + I1 + I2) * Rhat})')

The dimension of the true signal tensor: I1 x I2 x q: 12 x 16 x 10
Sample complexity for rank 3: O(114)


### Generate Phaseless Measurements

In [10]:
def initialize_zeros(dim):
    initializer = tf.zeros_initializer()
    return tf.Variable(initializer(shape=dim, dtype=tf.float32))

In [23]:
def generate_measurements(m, I1, I2, q):
    A = initialize_zeros([m, I1, I2, q]) # measurement tensor

    # Generate i.i.d. measurement tensors.
    for j in range(m):
        for k in range(q):
            # i.i.d. normal measurements from the independent number stream
            A[j,:,:,k].assign(tf.random.normal([I1, I2]))
    
    return A

start = time.time()
m = 200
A = generate_measurements(m, I1, I2, q)
print(f'Elapsed time: {time.time() - start} seconds.')

Elapsed time: 0.8982784748077393 seconds.


In [24]:
# Generate phaseless measurements.
Ylist = []
for k in range(q):
    # Normalize the true signal tensor with Frobenius norm.
    normalized_Xast = tf.linalg.normalize(Xast[:,:,k], ord='fro', axis=[0, 1])[0]
    Ylist.append(tf.tensordot(A[:,:,:,k], normalized_Xast, axes=([1, 2], [0, 1])))

Y = tf.reshape(tf.convert_to_tensor(Ylist), [m, q])
print(Y.shape)

(200, 10)


In [25]:
print('DEBUG')
print(f'\nDimension of the phaseless measurement matrix: m x q: {Y.shape[0]} x {Y.shape[1]}\n')
print('Phaseless Measurements:\n\n', Y)

DEBUG

Dimension of the phaseless measurement matrix: m x q: 200 x 10

Phaseless Measurements:

 tf.Tensor(
[[ 1.4173288   1.1490718   1.5035605  ... -0.0121095   0.15729125
  -1.039565  ]
 [-1.3361698  -0.22994685 -0.9057626  ... -0.38736638  0.618703
  -1.079539  ]
 [-1.4741212   2.2361112  -1.6096847  ...  0.92064166 -0.554445
  -1.8307835 ]
 ...
 [ 0.55836916  1.3080856  -1.2448782  ... -0.727911   -0.51140565
  -0.14848077]
 [-1.3519084  -1.7848707  -0.85040784 ... -0.11440651 -0.5337188
  -2.1890652 ]
 [-0.2648867   0.65368617 -0.52293766 ... -0.22146395 -0.5521327
   1.1964184 ]], shape=(200, 10), dtype=float32)


In [26]:
def initialize(I1, I2, q, R):
    """Initialize factor matrices."""

    U1 = tf.Variable(tf.random.normal([I1, R]))
    U2 = tf.Variable(tf.random.normal([I2, R]))
    B = tf.Variable(tf.random.normal([q, R]))

    return [U1, U2], B

In [27]:
def kruskal(U, B, R, Lambda=None, type='CP'):
    """Construct Tensor from Kruskal Formulation.

        Args:
            U: list consisting of two factor matrices U1 (I1 x R)
                and U2 (I2 x R) for the three-way case.
            B: the B (q x R) factor matrix.
            R: assumped rank (a scalar) of the low-rank tensor.
            Lambda: normalization factors (length R).
        
        Returns:
            Xhat: signal estimate (I1 x I2 x q).
    """
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # B = tf.cast(B, tf.float32)
    if type == 'CP':
        U1, U2 = U[0], U[1]
        I1, I2, q = U1.shape[0], U2.shape[0], B.shape[0]
        Xhat = tf.zeros([I1, I2, q])
        if Lambda is None:
            Lambda = tf.ones([R,])
        for r in range(R):
            U1U2 = tf.tensordot(U1[:, r], U2[:, r], axes=0)
            Xhat += Lambda[r] * tf.tensordot(U1U2, B[:, r], axes=0)
        
        return Xhat
    else:
        return None

In [28]:
def test_kruskal():
    warnings.filterwarnings("ignore", category=UserWarning)

    from tensorly.decomposition import parafac

    X = tl.tensor(np.arange(24).reshape((3, 4, 2)), dtype=tl.float32)
    test_rank = 4
    weights, factors = parafac(X, rank=test_rank)

    X_test = kruskal([factors[0], factors[1]], factors[2], test_rank, weights)

    err = X - X_test

    print('DEBUG | MSE for reconstruction: ', tf.math.reduce_sum(
        tf.multiply(err, err)).numpy())

test_kruskal() # test Kruskal tensor constructor



DEBUG | MSE for reconstruction:  3.6551285e-08


In [29]:
def reconstruct_error(Xast, U, B, R=10):
    err = Xast - kruskal(U, B, R)

    return tf.math.reduce_sum(tf.multiply(err, err)).numpy()

In [114]:
def descent(Uhat, Bhat, A, Y, R, max_iter):
    U1, U2 = Uhat[0], Uhat[1]
    I1, I2 = U1.shape[0], U2.shape[0]
    m = A.shape[0]
    q = Bhat.shape[0]

    @tf.function
    def solve_U1():
        """Helper function to solve the least squares 
            problem for factor matrix U1.
        """
        loss = 0
        m, I1, I2, q = A.shape
        R = Bhat.shape[1]
        vec_U1 = tf.reshape(U1, [I1 * R,])

        for k in range(q):
            yk = Y[:,k] # for linear projections
            Ak = A[:,:,:,k]
            bk = tf.reshape(Bhat[k,:], [1, R])

            # dim bk khatri_rao U2: R x I2
            U2B_kr = tf.transpose(tl.tenalg.khatri_rao([bk, U2]))
            A_kr = tl.tenalg.mode_dot(Ak, U2B_kr, 2)
            mat_A_kr = tf.reshape(A_kr, [m, I1 * R])
            
            yhat = tf.linalg.matvec(mat_A_kr, vec_U1)

            loss += (1 / m) * tf.math.reduce_sum(tf.square(yhat - yk))

        return loss
    
    @tf.function
    def solve_U2():
        """Helper function to solve the least squares 
            problem for factor matrix U2.
        """
        loss = 0
        m, I1, I2, q = A.shape
        R = Bhat.shape[1]

        vec_U2 = tf.reshape(U2, [I2 * R,])

        for k in range(q):
            yk = Y[:,k] # for linear projections
            Ak = tf.reshape(A[:,:,:,k], [m, I2, I1])
            bk = tf.reshape(Bhat[k,:], [1, R])

            # dim bk khatri_rao U1: R x I1
            U1B_KR = tf.transpose(tl.tenalg.khatri_rao([bk, U1]))
            A_kr = tl.tenalg.mode_dot(Ak, U1B_KR, 2)
            mat_A_kr = tf.reshape(A_kr, [m, I2 * R])

            yhat = tf.linalg.matvec(mat_A_kr, vec_U2)
            
            loss += (1 / m) * tf.math.reduce_sum(tf.square(yhat - yk))

        return loss
    
    '''
    Perform optimizations.
    '''

    opt = tf.keras.optimizers.Adam(learning_rate=0.1, beta_1=0.5, amsgrad=True)
    opt_iters = 100

    for t in range(max_iter):
        print(f'Iteration-{t}')
        print('Computing....')
        Xhat = kruskal(Uhat, Bhat, R)
        Cy = np.zeros([m, q]).astype('float32')

        '''
        Update Phase (for complex measurements only).
        '''
        # for k in range(q):
        #     AX = tf.tensordot(
        #         A[:,:,:,k], Xhat[:,:,k], axes=([1, 2], [0, 1]))
        #     Ck = tf.linalg.diag(tf.math.angle(AX))
        #     Cy[:, k] = tf.linalg.matvec(Ck, Y[:,k])
        
        # opt = tf.keras.optimizers.SGD(0.1)
        '''
        Solve for U1.
        '''
        for _ in range(opt_iters):
            opt.minimize(solve_U1, var_list=[U1])
        
        print('U1 optimized')
        # opt.minimize(solve_U1, var_list=[U1],tape=tf.GradientTape())

        '''
        Solve for U2.
        '''
        for _ in range(opt_iters):
            opt.minimize(solve_U2, var_list=[U2])
        
        print('U2 optimized')
        # opt.minimize(solve_U2, var_list=[U2],tape=tf.GradientTape())

        '''
        Solve for bk's.
        '''
        # def solve_B(q):
        #     least_squares_bks = []

        #     for i in range(q):
        #         @tf.function
        #         def solve_bk
        # vector = np.zeros((R,)).astype('float32')
        # rhs = tf.reduce_sum(Cy)
        # for k in range(q):
        #     bk = Bhat[k,:]
        #     U2U1_kr =  tl.tenalg.khatri_rao([U2, U1]) # khatri-rao product
        #     # for j in range(m):
        #     #     A3 = tf.reshape(A[j,:,:,k], [I1 * I2,])
        #     #     rhs += Cy[j, k]
        #     Ak = tf.reshape(A[:,:,:,k], [m, I1 * I2])
        #     vector += tf.math.reduce_sum(
        #         tf.linalg.matmul(Ak, U2U1_kr), axis=0)

        #     Bhat[k,:] = tf.multiply(vector, tf.math.reciprocal(rhs))
        
        # print(
        #     f'Reconstruction error: {reconstruct_error([U1, U2], Bhat, R):.2f}.')
    
    # Uhat = [U1, U2]
    
    return Uhat, Bhat

In [115]:
def plrpr(A, Y, R=5, max_iter=1):
    """Polyadic Low Rank Phase Retrieval.
    """
    Uinit, Binit = initialize(I1, I2, q, R)
    
    Uhat, Bhat = descent(Uinit, Binit, A, Y, R, max_iter)

    Xhat = kruskal(Uhat, Bhat, R)
    
    return Xhat

In [116]:
def test_plrpr(A, Y, Xast, R=10, max_iter=10):
    Xhat = plrpr(A, Y, R, max_iter)
    filename = FRAMES_DIR + video_name + '/frame-reconstructed-0' + '.jpg'
    cv2.imwrite(filename, Xhat[:,:,0].numpy())
    err = Xast - Xhat

    MSE = tf.math.reduce_sum(tf.multiply(err, err)).numpy()

    print(f'DEBUG | Mean squared error for reconstruction: {MSE}')

test_plrpr(A, Y, Xast, R = 10, max_iter = 1)


Iteration-0
Computing....
U1 optimized
U2 optimized
DEBUG | Mean squared error for reconstruction: 21033890.0


In [53]:
class TensorLRPR

SyntaxError: invalid syntax (<ipython-input-53-80ab800cc55c>, line 1)

In [ ]:
class TensorUtils

- Initialization (Spectral, HOSVD) for CP formulation.
- Complex measurements.
